# INCARs and job scripts templates for fireworks

In [6]:
INCAR_SCF = """PREC=accurate
ENCUT=500
NELMIN=10
EDIFF=1E-5
NSW=0
IBRION=-1
ISMEAR=0
SIGMA=0.01
ADDGRID=T
LWAVE=T
LCHARG=T
LREAL=F
LASPH=T
LORBIT=10

KGAMMA=T
KSPACING=0.15

NPAR=32
"""

INCAR_DOS = """PREC=accurate
ENCUT=500
NELMIN=10
EDIFF=1E-5
NSW=0
IBRION=-1
ISMEAR=-5
SIGMA=0.01
ADDGRID=T
LCHARG=F
LWAVE=F
LREAL=F
LASPH=T
LORBIT=10

NEDOS=2001
EMAX=20.00
EMIN=-20.00

KGAMMA=T
KSPACING=0.15

NPAR=32
"""

INCAR_BS = """PREC=accurate
ENCUT=500
NELMIN=10
EDIFF=1E-5
NSW=0
IBRION=-1
ISMEAR=0
SIGMA=0.01
ADDGRID=T
LWAVE=F
LCHARG=F
LREAL=F
LASPH=T
LORBIT=10

ICHARG=11
"""

job_sh_template = """#!/bin/bash
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=32
#SBATCH --partition=g1,g2
##
#SBATCH --job-name=zylu_demo
#SBATCH --time=99-24:59          # Runtime: Day-HH:MM
#SBATCH -o log.txt         # STDOUT
#SBATCH -e error.txt         # STDERR
#################################################################################

#################################################################################
VASP_BIN=/TGM/Apps/VASP/VASP_BIN/6.3.2/vasp.6.3.2.std.x
#################################################################################

hostname
date

#################################################################################
### Module Env
module purge
module add compiler/2022.1.0
module add mkl/2022.1.0
module add mpi/2021.6.0
export OMP_NUM_THREADS=1
export RSH_COMMAND="/usr/bin/ssh -x"
scontrol show hostname ${SLURM_JOB_NODELIST} | perl -ne 'chomb; print "$_"x1'> ${SLURM_SUBMIT_DIR}/nodelist
#################################################################################

mpirun -np $SLURM_NTASKS $VASP_BIN > stdout
"""

# POTCAR and POSCAR ready function

In [7]:
import os
from pathlib import Path
import shutil

def generate_potcar_from_poscar_flat(base_path):
    """
    Concatenates and generates a VASP POTCAR file from a flattened directory structure, based on the list of elements in the POSCAR file.

    Args:

    base_path (str): The root directory for the material calculation. The function assumes the POSCAR is located at base_path/1_scf/POSCAR.

    Returns:

    None. Prints status messages to the console.
    """
    # --- Path Definitions ---
    # The root directory of the POTCAR library, now points directly to your potcars folder.

    pot_path = Path('/home/student/zylu/Lu_potpaw_copy')
    base_path = Path(base_path)
    poscar_path = base_path / '1_scf' / 'POSCAR'
    output_path = base_path / '1_scf' / 'POTCAR'
    
    print(f"Generating POTCAR from {poscar_path} to {output_path}...")

    # --- Reading element symbols from POSCAR file ---
    try:
        with open(poscar_path, 'r') as f:
            lines = f.readlines()
        # In VASP 5+ format, element symbols are on the 6th line (index 5)
        elements = lines[5].strip().split()
        if not elements:
            print(f"Error: No element symbols found on line 6 of POSCAR file '{poscar_path}'.")
            return
        print(f"Elements found in POSCAR: {elements}")
    except FileNotFoundError:
        print(f"Error: File '{poscar_path}' not found.")
        return
    except IndexError:
        print(f"Error: Unable to read element line from '{poscar_path}'. It may not be a valid VASP 5+ format.")
        return

    # --- Concatenating POTCAR files ---
    try:
        with open(output_path, 'wb') as potcar_file:
            for element in elements:
                # *** Main modification part ***
                # Construct the path according to your 'POTCAR' filename format
                element_pot_path = pot_path / element / "POTCAR"

                if not os.path.exists(element_pot_path):
                    print(f"Error: POTCAR file for element '{element}' not found at '{element_pot_path}'")
                    # Clean up partially created file
                    potcar_file.close()
                    os.remove(output_path)
                    return

                with open(element_pot_path, 'rb') as element_file:
                    potcar_file.write(element_file.read())
        
        print(f"Successfully generated POTCAR for elements: {' '.join(elements)} at '{output_path}'")

    except Exception as e:
        (f"An unexpected error occurred during file writing: {e}")

def create_vasp_project(material_id, material_formula, base_path, POSCAR_content):

    base_dir = os.path.join(base_path, f"{material_id}_{material_formula}")
    scf_dir = os.path.join(base_dir, "1_scf")
    dos_dir = os.path.join(base_dir, "2_dos")
    bs_dir = os.path.join(base_dir, "3_bs") 

    # Create directories
    for d in [base_dir, scf_dir, dos_dir, bs_dir]:
        os.makedirs(d, exist_ok=True)

    # Create POSCAR, INCAR, job.sh in 1_scf directory
    # with open(os.path.join(scf_dir, "POSCAR"), "w") as f:
    #     f.write(f"{POSCAR_content}")
    target_poscar = os.path.join(scf_dir, "POSCAR")
    
    if os.path.exists(POSCAR_content) and os.path.isfile(POSCAR_content):
        # It's a file path, copy it
        shutil.copy(POSCAR_content, target_poscar)
    else:
        # It's a string content, write it
        with open(target_poscar, "w") as f:
            f.write(POSCAR_content)

    with open(os.path.join(scf_dir, "INCAR"), "w") as f:
        f.write(INCAR_SCF)
    with open(os.path.join(scf_dir, "job.sh"), "w") as f:
        f.write(job_sh_template)

    # Create INCAR in 2_dos 
    with open(os.path.join(dos_dir, "INCAR"), "w") as f:
        f.write(INCAR_DOS)

    # Create INCAR in 3_bs 
    with open(os.path.join(bs_dir, "INCAR"), "w") as f:
        f.write(INCAR_BS)
    
    generate_potcar_from_poscar_flat(base_path = base_dir)
    print(f"Project structure has been created for {material_id}_{material_formula}.")

In [1]:
from mp_api.client import MPRester
from emmet.core.symmetry import CrystalSystem

with MPRester("5ypNyStQsaDUA2neNLD17P8v2KNB241U") as mpr:
    
    docs = mpr.materials.summary.search(
        formula=["ABC3"],
        crystal_system=CrystalSystem.cubic,
    )
    print(f"found {len(docs)} ABX3 cubic materials.")


Retrieving SummaryDoc documents:   0%|          | 0/1538 [00:00<?, ?it/s]

found 1538 ABX3 cubic materials.


In [8]:
from fireworks import Firework, LaunchPad, FileTransferTask, PyTask, Workflow, ScriptTask
from fireworks.core.rocket_launcher import rapidfire
import Auto_baby
# from fireworks.queue.queue_launcher import rapidfire


launchpad = LaunchPad()
launchpad.reset('', require_password=False)

def Final_Calculation(material_id, formula, material_path: str = ''):
    if not material_path: raise ValueError("material_path must be provided.")
    

    _submit_SCF_Task_ = PyTask(func = "Auto_baby.scf_ready",
                               args=[material_path])


    

    _transfer_POSCAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/POSCAR', 
                                                                   'dest': f'{material_path}/2_dos/POSCAR'}], 'mode': 'copy'})
    _transfer_POTCAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/POTCAR', 
                                                       'dest': f'{material_path}/2_dos'}], 'mode': 'copy'})
    _transfer_JOB_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/job.sh', 
                                                  'dest': f'{material_path}/2_dos'}], 'mode': 'copy'})
    _transfer_WAVECAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/WAVECAR', 
                                                       'dest': f'{material_path}/2_dos'}], 'mode': 'copy'})
    _transfer_CONTCAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/CHGCAR', 
                                                                   'dest': f'{material_path}/2_dos/POSCCHGCARAR'}], 'mode': 'copy'})

    _submit_DOS_Task_ = PyTask(func= "Auto_baby.dos_ready",
                               args=[material_path])



    _transfer_CONTCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/CHGCAR', 
                                                    'dest': f'{material_path}/3_bs/POSCAR'}], 'mode': 'copy'})
    _transfer_POTCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/POTCAR', 
                                                    'dest': f'{material_path}/3_bs'}], 'mode': 'copy'})
    _transfer_JOB_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/job.sh', 
                                                    'dest': f'{material_path}/3_bs'}], 'mode': 'copy'})
    _transfer_CHGCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/CHGCAR', 
                                                    'dest': f'{material_path}/3_bs'}], 'mode': 'copy'})
    

    _submit_BS_Task_ = PyTask(func = "Auto_baby.bs_ready",
                             args=[material_path])

    


    scf_calculation = Firework(
        tasks=[_submit_SCF_Task_],
        name="SCF Calculation" 
    )

    DOS_calculation = Firework(
        tasks=[_transfer_POSCAR_of_SCF_to_DOS, _transfer_CONTCAR_of_SCF_to_DOS, _transfer_POTCAR_of_SCF_to_DOS, _transfer_JOB_of_SCF_to_DOS, _transfer_WAVECAR_of_SCF_to_DOS,_submit_DOS_Task_],
        name="DOS Calculation",
        parents=[scf_calculation]
    )

    BAND_structure_calculation = Firework(
        tasks=[_transfer_CONTCAR_of_SCF_to_BS, _transfer_POTCAR_of_SCF_to_BS, _transfer_JOB_of_SCF_to_BS, _transfer_CHGCAR_of_SCF_to_BS, _submit_BS_Task_],
        name="Band Structure Calculation",
        parents=[scf_calculation]
    )



    wf = Workflow([scf_calculation, DOS_calculation, BAND_structure_calculation], name = f"{material_id}_{formula}")
    launchpad.add_wf(wf)


2025-11-24 14:24:07,074 INFO Performing db tune-up
2025-11-24 14:24:07,083 INFO LaunchPad was RESET.


In [9]:
materials_base_path = "/home/student/zylu/materials_demo"

for doc in docs[:1]:
    structure = doc.structure
    formula = doc.formula_pretty
    material_id = doc.material_id

    path = Path(materials_base_path) / f"{material_id}_{formula}"

    create_vasp_project(material_id, formula, base_path=materials_base_path, POSCAR_content=structure.to(fmt="poscar"))
    

    Final_Calculation(material_path=path, material_id = material_id, formula = formula)



Generating POTCAR from /home/student/zylu/materials_demo/mp-1183115_AcAlO3/1_scf/POSCAR to /home/student/zylu/materials_demo/mp-1183115_AcAlO3/1_scf/POTCAR...
Elements found in POSCAR: ['Ac', 'Al', 'O']
Successfully generated POTCAR for elements: Ac Al O at '/home/student/zylu/materials_demo/mp-1183115_AcAlO3/1_scf/POTCAR'
Project structure has been created for mp-1183115_AcAlO3.
2025-11-24 14:24:09,030 INFO Added a workflow. id_map: {-3: 1, -2: 2, -1: 3}


In [10]:
rapidfire(launchpad)

2025-11-24 14:24:12,091 INFO Created new dir /home/student/zylu/05_fireworks/launcher_2025-11-24-05-24-12-091756
2025-11-24 14:24:12,092 INFO Launching Rocket
2025-11-24 14:24:12,099 INFO RUNNING fw_id: 3 in directory: /home/student/zylu/05_fireworks/launcher_2025-11-24-05-24-12-091756
2025-11-24 14:24:12,103 INFO Task started: PyTask.
All required calculation inputs and job scripts are present where needed.
Current SLURM job state: UNKNOWN
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: COMPLETED
Final SLURM job state: COMPLETED
2025-11-24 14:24:22,051 INFO Task completed: PyTask
2025-11-24 14:24:22,063 INFO Rocket finished
2025-11-24 14:24:22,065 INFO Created new dir /home/student/zylu/05_fireworks/launcher_2025-11-24-05-24-22-065512
2025-11-24 14

/home/student/Programs/anaconda3/envs/fireworks/lib/python3.14/site-packages/pymatgen/core/structure.py:3087: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
/home/student/Programs/anaconda3/envs/fireworks/lib/python3.14/site-packages/pymatgen/io/vasp/inputs.py:1629: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="wt") as file:


Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: COMPLETED
Final SLURM job state: COMPLETED
2025-11-24 14:24:52,725 INFO Task complet